This is the processing code to generate Multi-omics data and pcHi-C evidence graph

In [1]:
import functools
import itertools
import os

import anndata
import networkx as nx
import numpy as np
import pandas as pd
import scanpy as sc
from matplotlib import rcParams
from networkx.algorithms.bipartite import biadjacency_matrix

import scglue

#### Read Data and filtering cell types

Cell type filtering was done here to keep consistent with pcHi-C data.

In [2]:
rna = anndata.read_h5ad("../../fig2/PBMC_data/data/PBMC_10X_GEX.h5ad")
rna.X = rna.layers['counts'].copy()
rna


AnnData object with n_obs × n_vars = 10412 × 36601
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'seurat_annotations', 'RNA_snn_res.0.4', 'seurat_clusters', 'RNA_snn_res.0.6', 'RNA_snn_res.0.8', 'RNA_snn_res.1', 'RNA_snn_res.0.9'
    var: 'mean', 'std'
    layers: 'counts'

In [3]:
atac = anndata.read_h5ad("../../fig2/PBMC_data/data/PBMC_10X_ATAC.peak.h5ad")
atac


AnnData object with n_obs × n_vars = 10412 × 106056
    obs: 'orig.ident', 'nCount_ATAC', 'nFeature_ATAC', 'seurat_annotations', 'nCount_ACTIVITY', 'nFeature_ACTIVITY', 'ATAC_snn_res.0.8', 'seurat_clusters'

In [4]:
np.all(atac.obs_names == rna.obs_names)

True

In [5]:
rna.obs["seurat_annotations"].cat.categories

Index(['CD4 Naive', 'CD4 TCM', 'CD4 TEM', 'CD8 Naive', 'CD8 TEM_1',
       'CD8 TEM_2', 'CD14 Mono', 'CD16 Mono', 'HSPC', 'Intermediate B', 'MAIT',
       'Memory B', 'NK', 'Naive B', 'Plasma', 'Treg', 'cDC', 'gdT', 'pDC'],
      dtype='object')

In [6]:
used_cts = {
    "CD4 Naive", "CD4 TCM", "CD4 TEM", "CD8 Naive", "CD8 TEM_1", "CD8 TEM_2",
    "CD14 Mono", "CD16 Mono", "Memory B", "Naive B"
}  # To match cell types covered in PC Hi-C
used_chroms = {f"chr{x}" for x in range(1, 23)}.union({"chrX"})

#### Selecting features in known chromatin

In [7]:
ref_tss = pd.read_csv('../utils/hg38_ref_TSS.txt', sep='\t')
ref_tss.index = ref_tss['gene_name']
ref_tss = ref_tss.drop_duplicates(subset=['symbol'], keep='first')
non_exist_gene = list(set(rna.var.index.tolist()) - set(ref_tss.index.tolist()))
non_df = pd.DataFrame(np.zeros((len(non_exist_gene),11)))
non_df.index = non_exist_gene
non_df.columns = ref_tss.columns
#print(non_df.shape)
#print(ref_tss.shape)
ref_tss = pd.concat([ref_tss, non_df])
ref_tss = ref_tss.loc[rna.var.index,:]
#print(ref_tss.shape)
np.all(ref_tss.index.to_numpy() == rna.var.index.to_numpy())

True

In [8]:
rna.var['chrom'] = ref_tss['seqnames'].to_numpy().astype(str)
rna.var['chromStart'] = ref_tss['start'].to_numpy()
rna.var['chromEnd'] = ref_tss['end'].to_numpy()
rna.var['strand'] = ref_tss['strand'].to_numpy()


In [9]:
rna = rna[
    [item in used_cts for item in rna.obs["seurat_annotations"]],
    [item in used_chroms for item in rna.var["chrom"]]
]
sc.pp.filter_genes(rna, min_counts=1)
rna.obs_names += "-RNA"
rna

Trying to set attribute `.var` of view, copying.


AnnData object with n_obs × n_vars = 8798 × 16376
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'seurat_annotations', 'RNA_snn_res.0.4', 'seurat_clusters', 'RNA_snn_res.0.6', 'RNA_snn_res.0.8', 'RNA_snn_res.1', 'RNA_snn_res.0.9'
    var: 'mean', 'std', 'chrom', 'chromStart', 'chromEnd', 'strand', 'n_counts'
    layers: 'counts'

In [10]:
atac.var['chrom'] = atac.var.index.str.split('-').str[0]
atac.var['chromStart'] = atac.var.index.str.split('-').str[1]
atac.var['chromEnd'] = atac.var.index.str.split('-').str[2]


In [11]:
atac

AnnData object with n_obs × n_vars = 10412 × 106056
    obs: 'orig.ident', 'nCount_ATAC', 'nFeature_ATAC', 'seurat_annotations', 'nCount_ACTIVITY', 'nFeature_ACTIVITY', 'ATAC_snn_res.0.8', 'seurat_clusters'
    var: 'chrom', 'chromStart', 'chromEnd'

In [12]:
atac = atac[
    [item in used_cts for item in atac.obs["seurat_annotations"]],
    [item in used_chroms for item in atac.var["chrom"]]
]
sc.pp.filter_genes(atac, min_counts=1)
atac.obs_names += "-ATAC"
atac

Trying to set attribute `.var` of view, copying.


AnnData object with n_obs × n_vars = 8798 × 106052
    obs: 'orig.ident', 'nCount_ATAC', 'nFeature_ATAC', 'seurat_annotations', 'nCount_ACTIVITY', 'nFeature_ACTIVITY', 'ATAC_snn_res.0.8', 'seurat_clusters'
    var: 'chrom', 'chromStart', 'chromEnd', 'n_counts'

In [13]:
np.all(atac.obs_names.str.split('-').str[0] == rna.obs_names.str.split('-').str[0])

True

In [14]:
genes = scglue.genomics.Bed(rna.var.assign(name=rna.var_names))
peaks = scglue.genomics.Bed(atac.var.assign(name=atac.var_names))
tss = genes.strand_specific_start_site()
promoters = tss.expand(2000, 0)

In [15]:
rna.X = rna.layers["counts"].copy()
sc.pp.highly_variable_genes(rna, n_top_genes=6000,flavor="seurat_v3")
sc.pp.normalize_total(rna)
sc.pp.log1p(rna)
sc.pp.scale(rna, max_value=10)
sc.tl.pca(rna, n_comps=100, use_highly_variable=True, svd_solver="auto")
sc.pp.neighbors(rna, n_pcs=100)
sc.tl.umap(rna)

In [16]:
rna.X = rna.layers["counts"].copy()

In [17]:
#atac.X = atac.layers['counts'].copy()
scglue.data.lsi(atac, n_components=100, use_highly_variable=False, n_iter=15)
sc.pp.neighbors(atac, n_pcs=100, use_rep="X_lsi")
sc.tl.umap(atac)

/mnt/hdd/chaozhong/conda/miniconda3/lib/python3.7/site-packages/sklearn/utils/extmath.py:376: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  FutureWarning,


### Build Graph to overlap with pcHi-C evidence

#### Overlap

In [18]:
overlap_graph = scglue.genomics.window_graph(
    genes.expand(2000, 0), peaks, 0,
    attr_fn=lambda l, r, d: {
        "weight": 1.0,
        "type": "overlap"
    }
)
overlap_graph = nx.DiGraph(overlap_graph)
overlap_graph.number_of_edges()

  0%|          | 0/16376 [00:00<?, ?it/s]

13642

#### Genomic distance

In [19]:
dist_graph = scglue.genomics.window_graph(
    promoters, peaks, 150000,
    attr_fn=lambda l, r, d: {
        "dist": abs(d),
        "weight": scglue.genomics.dist_power_decay(abs(d)),
        "type": "dist"
    }
)
dist_graph = nx.DiGraph(dist_graph)
dist_graph.number_of_edges()


  0%|          | 0/16376 [00:00<?, ?it/s]

381345

#### pcHi-C

In [21]:
frags = pd.read_table(
    "../pcHiC/Human_hg38/Digest_Human_HindIII.rmap",
    header=None, names=["chrom", "chromStart", "chromEnd", "name"],
    dtype={"name": str}
)
frags["chromStart"] -= 1  # Originally 1-based, convert to 0-based as in BED
frags.index = frags["name"]
frags.head(n=3)

,chrom,chromStart,chromEnd,name
name,,,,
2,chr1,16007,24571,2
3,chr1,24571,27981,3
4,chr1,27981,30429,4


In [22]:
baits = pd.read_table(
    "../pcHiC/Human_hg38/Digest_Human_HindIII_baits_e75_ID.baitmap",
    header=None, names=["chrom", "chromStart", "chromEnd", "name", "targets"],
    usecols=["name"], dtype={"name": str}
)["name"].to_numpy()
baits = scglue.genomics.Bed(frags.loc[baits, :])

In [23]:
used_cts = ["Mon", "nCD4", "tCD4", "aCD4", "naCD4", "nCD8", "tCD8", "nB", "tB"]
bait_oe = pd.read_table(
    "../pcHiC/PCHiC_peak_matrix_cutoff5.tsv",
    usecols=["baitID", "oeID"] + used_cts, dtype={"baitID": str, "oeID": str}
)
bait_oe = bait_oe.loc[bait_oe.loc[:, used_cts].to_numpy().max(axis=1) > 5, ["baitID", "oeID"]]
bait_oe.shape

(517191, 2)

In [25]:
frags_set, baits_set = set(frags["name"]), set(baits["name"])
bait_oe = bait_oe.loc[[
    i in baits_set and j in frags_set
    for i, j in zip(bait_oe["baitID"], bait_oe["oeID"])
], :]  # Some frags might be missing if liftover is used
bait_oe.shape

(516753, 2)

In [26]:
bait_oe = pd.concat([bait_oe, pd.DataFrame({"baitID": baits.index, "oeID": baits.index})])  # Add same-frag links
bait_oe = nx.from_pandas_edgelist(bait_oe, source="baitID", target="oeID", create_using=nx.DiGraph)
oes = scglue.genomics.Bed(frags.loc[np.unique([e[1] for e in bait_oe.edges]), :])

In [27]:
gene_bait = scglue.genomics.window_graph(promoters, baits, 1000)
oe_peak = scglue.genomics.window_graph(oes, peaks, 1000)

  0%|          | 0/16376 [00:00<?, ?it/s]

  0%|          | 0/192420 [00:00<?, ?it/s]

In [28]:
pchic_graph = (
    biadjacency_matrix(gene_bait, genes.index, baits.index, weight=None) @
    biadjacency_matrix(bait_oe, baits.index, oes.index, weight=None) @
    biadjacency_matrix(oe_peak, oes.index, peaks.index, weight=None)
).tocoo()
pchic_graph.eliminate_zeros()
pchic_graph.data = np.minimum(pchic_graph.data, 1.0)
pchic_graph = nx.DiGraph([
    (genes.index[i], peaks.index[j], {"weight": k, "type": "pchic"})
    for i, j, k in zip(pchic_graph.row, pchic_graph.col, pchic_graph.data)
])
pchic_graph.number_of_edges()

255985

In [29]:
pchic_links = nx.to_pandas_edgelist(
    bait_oe, source="baitID", target="oeID"
).query(
    "baitID != oeID"
).merge(
    frags, how="left", left_on="baitID", right_index=True
).merge(
    frags, how="left", left_on="oeID", right_index=True
).merge(
    nx.to_pandas_edgelist(gene_bait, source="gene", target="baitID"), how="left", on="baitID"
).dropna(subset=["gene"]).assign(score=1).loc[:, [
    "chrom_x", "chromStart_x", "chromEnd_x",
    "chrom_y", "chromStart_y", "chromEnd_y",
    "score", "gene"
]]
pchic_links = pchic_links.query("chrom_x == chrom_y")
pchic_links.to_csv(f"pchic.annotated_links", sep="\t", index=False, header=False)

In [30]:
nx.write_graphml(pchic_graph, "pchic.graphml.gz")

In [31]:
rna.var["in_pchic"] = biadjacency_matrix(gene_bait, genes.index).sum(axis=1).A1 != 0
rna.var["in_pchic"].sum()

14805

### Update highly-variable genes

In [32]:
rna.var["o_highly_variable"] = rna.var["highly_variable"]
rna.var["o_highly_variable"].sum()

6000

In [33]:
rna.var["in_cicero"] = biadjacency_matrix(
    scglue.genomics.window_graph(promoters, peaks, 0),
    genes.index
).sum(axis=1).A1 > 0
rna.var["in_cicero"].sum()

  0%|          | 0/16376 [00:00<?, ?it/s]

12763

In [34]:
rna.var["d_highly_variable"] = functools.reduce(np.logical_and, [
    rna.var["highly_variable"],
    rna.var["in_pchic"],
    rna.var["in_cicero"]
])
rna.var["d_highly_variable"].sum()

4342

In [35]:
rna.var["dcq_highly_variable"] = rna.var["highly_variable"]
rna.var["dcq_highly_variable"].sum()

6000

In [36]:
o_prior = overlap_graph.copy()

In [37]:
hvg_reachable = scglue.graph.reachable_vertices(o_prior, rna.var.query("o_highly_variable").index)


In [38]:
atac.var["o_highly_variable"] = [item in hvg_reachable for item in atac.var_names]
atac.var["o_highly_variable"].sum()

4833

In [39]:
o_prior = scglue.graph.compose_multigraph(o_prior, o_prior.reverse())
for item in itertools.chain(atac.var_names, rna.var_names):
    o_prior.add_edge(item, item, weight=1.0, type="self-loop")
nx.set_edge_attributes(o_prior, 1, "sign")

In [40]:
o_prior = o_prior.subgraph(hvg_reachable)

In [41]:
d_prior = dist_graph.copy()

In [42]:
hvg_reachable = scglue.graph.reachable_vertices(d_prior, rna.var.query("d_highly_variable").index)


In [43]:
atac.var["d_highly_variable"] = [item in hvg_reachable for item in atac.var_names]
atac.var["d_highly_variable"].sum()

63169

In [ ]:
d_prior = scglue.graph.compose_multigraph(d_prior, d_prior.reverse())
for item in itertools.chain(atac.var_names, rna.var_names):
    d_prior.add_edge(item, item, weight=1.0, type="self-loop")
nx.set_edge_attributes(d_prior, 1, "sign")

In [ ]:
d_prior = d_prior.subgraph(hvg_reachable)


In [ ]:
rna.write(f"rna.h5ad", compression="gzip")
atac.write(f"atac.h5ad", compression="gzip")